# 2. Feature Engineering and Data Merging

## 2.1. Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
# Statistics
df_stats = pd.read_csv('./Data/Statistics/Seasons_Stats.csv',index_col=0)
df_finish = pd.read_csv('./Data/Statistics/player_data.csv')

In [3]:
# Career statistics
df_career_stats = pd.read_csv('./Data/career_stats.csv')

In [4]:
# Other information
df_coach_awards = pd.read_csv('./Data/Other/basketball_awards_coaches.csv')
df_player_awards = pd.read_csv('./Data/Other/basketball_awards_players.csv')
df_coaches = pd.read_csv('./Data/Other/basketball_coaches.csv')
df_draft = pd.read_csv('./Data/Other/basketball_draft.csv')
df_hof = pd.read_csv('./Data/Other/basketball_hof.csv')
df_info = pd.read_csv('./Data/Other/basketball_master.csv')
df_all_star = pd.read_csv('./Data/Other/basketball_player_allstar.csv')
df_teams = pd.read_csv('./Data/Other/basketball_teams.csv')

## 2.2. Feature Engineering - Coach Awards

In [5]:
df_coach_awards['award'].value_counts()

NBA Coach of the Year    50
ABA Coach of the Year    11
Name: award, dtype: int64

Here, we have two types of awards: one for the NBA and one for the ABA. Let us calculate the sum of awards won by each award-winning coach. 

In [6]:
df_coach_awards_count = pd.DataFrame(df_coach_awards.groupby('coachID').count()['award'])

In [7]:
df_coach_awards_count.columns = ['award_count']
df_coach_awards_count = df_coach_awards_count.reset_index()

In [8]:
df_coach_awards_count.head()

,coachID,award_count
0,auerbre01,1
1,belmojo01,1
2,biancal01,1
3,birdla01,1
4,brooksc01,1


## 2.3. Feature Engineering - Player Awards

In [9]:
df_player_awards['award'].value_counts()

All-NBA First Team                     331
All-NBA Second Team                    330
All-Defensive First Team               226
All-Defensive Second Team              225
All-NBA Third Team                     120
Rookie of the Year                      85
Most Valuable Player                    80
Finals MVP                              50
All-ABA Second Team                     46
All-ABA First Team                      45
Defensive Player of the Year            30
Sixth Man of the Year                   30
All-Rookie First Team                   27
Most Improved Player                    27
All-Rookie Second Team                  26
All-Defensive Team                      22
Comeback                                 6
Sportsmanship Award                      5
J. Walter Kennedy Citizenship Award      4
Finals MVP                               3
Executive of the Year                    1
Name: award, dtype: int64

It would not be very wise to keep a separate count of each award for all players. We should determine how to group the awards so that the number of award features in our model is not excessive. Here is how we will proceed:
- all_team_awards: All-NBA First Team, All-NBA Second Team, All-Defensive First Team, All-Defensive Second Team, All-NBA Third Team, All-ABA Second Team, All-ABA First Team, All-Rookie First Team, All-Rookie Second Team, All-Defensive Team
- league_awards: Rookie of the Year, Most Valuable Player, Finals MVP, Sixth-Man of the Year, Defensive Player of the Year, Most Improved Player

We are dropping the remaining awards because their counts are so few. 

This grouping is certainly not the optimal grouping for predictive results, but it still remains logical. Perhaps, these groupings could be adjusted in the future. 

In [10]:
df_player_awards['award'].unique()

array(['All-NBA First Team', 'All-NBA Second Team', 'Rookie of the Year',
       'Most Valuable Player', 'All-ABA First Team',
       'All-ABA Second Team', 'Finals MVP', 'All-Defensive First Team',
       'All-Defensive Second Team', 'All-Defensive Team', 'Comeback',
       'Defensive Player of the Year', 'Sixth Man of the Year',
       'Most Improved Player', 'All-NBA Third Team',
       'Executive of the Year', 'All-Rookie First Team',
       'Sportsmanship Award', 'All-Rookie Second Team', 'Finals MVP ',
       'J. Walter Kennedy Citizenship Award'], dtype=object)

In [11]:
all_team_awards = ['All-NBA First Team','All-NBA Second Team','All-ABA First Team','All-ABA Second Team','All-Defensive First Team','All-Defensive Second Team','All-Defensive Team','All-NBA Third Team','All-Rookie First Team','All-Rookie Second Team']
all_league_awards = ['Rookie of the Year','Most Valuable Player','Finals MVP','Defensive Player of the Year','Sixth Man of the Year','Most Improved Player','Finals MVP ']

In [12]:
awards = []
for player in sorted(df_player_awards['playerID'].unique()):
    team_count = 0
    league_count = 0
    for award in df_player_awards[df_player_awards['playerID']==player]['award']:
        if (award in all_team_awards):
            team_count = team_count+1
        elif(award in all_league_awards):
            league_count = league_count+1
    awards.append((team_count,league_count))

In [13]:
awards[:5]

[(26, 9), (0, 1), (0, 1), (0, 0), (1, 0)]

In [14]:
awards_dict = dict(zip(sorted(df_player_awards['playerID'].unique()),awards))

In [15]:
df_player_awards_count = pd.DataFrame({'playerID':df_player_awards['playerID'].unique()})

df_player_awards_count['all_team_awards'] = df_player_awards_count['playerID'].map(awards_dict)
df_player_awards_count['league_awards'] = df_player_awards_count['playerID'].map(awards_dict)

df_player_awards_count['all_team_awards'] = df_player_awards_count['all_team_awards'].apply(lambda x:x[0])
df_player_awards_count['league_awards'] = df_player_awards_count['league_awards'].apply(lambda x:x[1])

In [16]:
df_player_awards_count.head()

,playerID,all_team_awards,league_awards
0,feeribo01,3,0
1,fulksjo01,4,0
2,mckinho01,2,0
3,miasest01,2,0
4,zasloma01,4,0


## 2.4. Feature Engineering - Coaches

Former NBA players who continue their basketball careers in NBA coaching could be more likely to make the hall of fame. We will add a coaching feature that indicates many years an NBA player has coached in the league. 

In [17]:
coaches = []
players = sorted(df_info['bioID'].unique())
for coach in df_coaches['coachID'].unique():
    if (coach in players):
        coaches.append((coach,len(df_coaches[df_coaches['coachID']==coach])))

In [18]:
df_coaches_years = pd.DataFrame({'playerID':players})
df_coaches_years['years_coaching'] = df_coaches_years['playerID'].map(dict(coaches))

In [19]:
df_coaches_years.head()

,playerID,years_coaching
0,abdelal01,NaN
1,abdulka01,NaN
2,abdulma01,NaN
3,abdulma02,NaN
4,abdulta01,NaN


In [20]:
df_coaches_years['years_coaching'] = df_coaches_years['years_coaching'].fillna(0)

In [21]:
df_coaches_years.head()

,playerID,years_coaching
0,abdelal01,0.0
1,abdulka01,0.0
2,abdulma01,0.0
3,abdulma02,0.0
4,abdulta01,0.0


In [22]:
df_coaches_years['years_coaching'] = df_coaches_years['years_coaching'].astype(int)

In [23]:
df_coaches_years.head()

,playerID,years_coaching
0,abdelal01,0
1,abdulka01,0
2,abdulma01,0
3,abdulma02,0
4,abdulta01,0


## 2.5. Feature Engineering - Draft

The most talented NBA players are usually selected as lottery picks in the NBA draft. Lottery picks consist of the first overall pick to the 14th overall pick. We will create a binary draft feature that determines if players were drafted in the lottery. 

In [24]:
# Exclude ABA draft selections to avoid duplicates
df_draft_lottery = df_draft[df_draft['lgID']=='NBA']

In [25]:
df_draft_lottery = df_draft_lottery[['playerID','draftOverall']]

In [26]:
df_draft_lottery.head()

,playerID,draftOverall
1444,NaN,0
1445,NaN,0
1446,gallaha01,0
1447,NaN,0
1448,gaineel01,0


In [27]:
def draft_transform(pick):
    if ((pick<15) & (pick!=0)):
        return 1
    else:
        return 0

In [28]:
df_draft_lottery['lottery'] = df_draft_lottery['draftOverall'].apply(draft_transform)

In [29]:
df_draft_lottery = df_draft_lottery.drop('draftOverall',axis=1)

In [30]:
df_draft_lottery.head()

,playerID,lottery
1444,NaN,0
1445,NaN,0
1446,gallaha01,0
1447,NaN,0
1448,gaineel01,0


In [31]:
df_draft[df_draft['playerID']=='gallaha01']

,draftYear,draftRound,draftSelection,draftOverall,tmID,firstName,lastName,suffixName,playerID,draftFrom,lgID
1446,1947,0,0,0,BLB,Harry,Gallatin,NaN,gallaha01,NE Missouri State Teachers College,NBA
1580,1948,0,0,0,NYK,Harry,Gallatin,NaN,gallaha01,NE Missouri State,NBA


It seems that pre-modern players were able to be drafted multiple times. We will exclude these duplicate values as well. 

In [32]:
df_draft_lottery = df_draft_lottery[df_draft_lottery['playerID'].duplicated()==False]

## 2.6. Feature Engineering - Hall of Fame

We just need to create a binary feature that tells whether or not a player in the hall of fame.
- 1 = in hall of fame
- 0 = not in hall of fame

In [33]:
hof = df_info['bioID'].apply(lambda x:x in df_hof['hofID'].unique()).map({True:1,False:0})
df_hof_indicator = pd.DataFrame({'bioID':df_info['bioID'],'hof':hof})

In [34]:
df_hof_indicator.head()

,bioID,hof
0,abdelal01,0
1,abdulka01,1
2,abdulma01,0
3,abdulma02,0
4,abdulta01,0


## 2.7. Feature Engineering - General Information

In [35]:
df_info.columns

Index(['bioID', 'useFirst', 'firstName', 'middleName', 'lastName', 'nameGiven',
       'fullGivenName', 'nameSuffix', 'nameNick', 'pos', 'firstseason',
       'lastseason', 'height', 'weight', 'college', 'collegeOther',
       'birthDate', 'birthCity', 'birthState', 'birthCountry', 'highSchool',
       'hsCity', 'hsState', 'hsCountry', 'deathDate', 'race'],
      dtype='object')

In [36]:
df_info_new = df_info[['bioID','useFirst','nameNick','firstName','lastName','height','weight','college','birthCountry','race']]

In [37]:
df_info_new

,bioID,useFirst,nameNick,firstName,lastName,height,weight,college,birthCountry,race
0,abdelal01,Alaa,NaN,Alaa,Abdelnaby,82.0,240.0,Duke,EGY,B
1,abdulka01,Kareem,"Lew, Cap",Kareem,Abdul-Jabbar,85.0,225.0,UCLA,USA,B
2,abdulma01,Mahdi,Walt,Mahdi,Abdul-Rahman,74.0,185.0,UCLA,USA,B
3,abdulma02,Mahmoud,NaN,Mahmoud,Abdul-Rauf,73.0,162.0,Louisiana State,USA,B
4,abdulta01,Tariq,NaN,Tariq,Abdul-Wahad,78.0,223.0,San Jose State,FRA,B
...,...,...,...,...,...,...,...,...,...,...
5056,rosegl01,Glen,NaN,Glen,Rose,NaN,NaN,NaN,NaN,NaN
5057,shephle01,Len,NaN,Len,Shepherd,NaN,NaN,NaN,NaN,NaN
5058,glammge01,George,NaN,George,Glammack,NaN,NaN,NaN,NaN,NaN
5059,eurasge01,Gene,NaN,Gene,Eurash,NaN,NaN,NaN,NaN,NaN


The features above are the ones with which we will be working. We should notice that there are rows with nearly all null values, signifying that those rows are not players. We will filter them out during the merging stage. 

1. We will create a binary nickname feature. 

In [38]:
def nick_transform(nickname):
    if (pd.isnull(nickname)):
        return 0
    else:
        return 1

In [39]:
df_info_new['nickname'] = df_info_new['nameNick'].apply(nick_transform)

C:\Users\lucas\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [40]:
df_info_new.head()

,bioID,useFirst,nameNick,firstName,lastName,height,weight,college,birthCountry,race,nickname
0,abdelal01,Alaa,NaN,Alaa,Abdelnaby,82.0,240.0,Duke,EGY,B,0
1,abdulka01,Kareem,"Lew, Cap",Kareem,Abdul-Jabbar,85.0,225.0,UCLA,USA,B,1
2,abdulma01,Mahdi,Walt,Mahdi,Abdul-Rahman,74.0,185.0,UCLA,USA,B,1
3,abdulma02,Mahmoud,NaN,Mahmoud,Abdul-Rauf,73.0,162.0,Louisiana State,USA,B,0
4,abdulta01,Tariq,NaN,Tariq,Abdul-Wahad,78.0,223.0,San Jose State,FRA,B,0


2. We will create a binary "top college" feature. The 25 colleges with the most NBA players will be considered a top college. 

In [41]:
df_info_new['college'].value_counts().head(26)

none                    295
Kentucky                 83
UCLA                     80
Notre Dame               71
Indiana                  67
North Carolina           64
Kansas                   59
St. John's (NY)          59
Duke                     56
DePaul                   53
Ohio State               51
Illinois                 50
Louisville               48
Michigan                 48
Minnesota                47
Marquette                47
Purdue                   45
Wisconsin                43
Michigan State           42
North Carolina State     41
Arizona                  41
Syracuse                 40
Villanova                40
Maryland                 39
Georgia Tech             39
Temple                   38
Name: college, dtype: int64

In [42]:
top_schools = df_info_new['college'].value_counts().head(26).index[1:] # "none" dropped

In [43]:
def college_transform(college):
    if (college in top_schools):
        return 1
    else:
        return 0

In [44]:
df_info_new['top_college'] = df_info_new['college'].apply(college_transform)

C:\Users\lucas\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [45]:
df_info_new.head()

,bioID,useFirst,nameNick,firstName,lastName,height,weight,college,birthCountry,race,nickname,top_college
0,abdelal01,Alaa,NaN,Alaa,Abdelnaby,82.0,240.0,Duke,EGY,B,0,1
1,abdulka01,Kareem,"Lew, Cap",Kareem,Abdul-Jabbar,85.0,225.0,UCLA,USA,B,1,1
2,abdulma01,Mahdi,Walt,Mahdi,Abdul-Rahman,74.0,185.0,UCLA,USA,B,1,1
3,abdulma02,Mahmoud,NaN,Mahmoud,Abdul-Rauf,73.0,162.0,Louisiana State,USA,B,0,0
4,abdulta01,Tariq,NaN,Tariq,Abdul-Wahad,78.0,223.0,San Jose State,FRA,B,0,0


3. We will create a binary foreign-birth feature. 

In [46]:
def country_transform(country):
    if ((country=='USA') | (pd.isna(country))): # we will assume null birth countries are not foreign (based on sample analysis)
        return 0
    else:
        return 1

In [47]:
df_info_new['foreign_birth'] = df_info_new['birthCountry'].apply(country_transform)

C:\Users\lucas\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [48]:
df_info_new.head()

,bioID,useFirst,nameNick,firstName,lastName,height,weight,college,birthCountry,race,nickname,top_college,foreign_birth
0,abdelal01,Alaa,NaN,Alaa,Abdelnaby,82.0,240.0,Duke,EGY,B,0,1,1
1,abdulka01,Kareem,"Lew, Cap",Kareem,Abdul-Jabbar,85.0,225.0,UCLA,USA,B,1,1,0
2,abdulma01,Mahdi,Walt,Mahdi,Abdul-Rahman,74.0,185.0,UCLA,USA,B,1,1,0
3,abdulma02,Mahmoud,NaN,Mahmoud,Abdul-Rauf,73.0,162.0,Louisiana State,USA,B,0,0,0
4,abdulta01,Tariq,NaN,Tariq,Abdul-Wahad,78.0,223.0,San Jose State,FRA,B,0,0,1


4. We will prepare the race column to be separated into three categories: black, white, and other. These values will be represented in the form of dummy variables for our model.

In [49]:
df_info_new['race'].value_counts()

B    2460
W    2431
O       9
1       2
Name: race, dtype: int64

In [50]:
df_info_new[df_info_new['race']=='1']

,bioID,useFirst,nameNick,firstName,lastName,height,weight,college,birthCountry,race,nickname,top_college,foreign_birth
146,biedran01,Andris,NaN,Andris,Biedrins,83.0,240.0,none,LAT,1,0,0,1
3058,millemi01,Mike,Mike,Michael,Miller,80.0,211.0,Florida,USA,1,1,0,0


Both of the players listed above are white. We will adjust their race values. 

In [51]:
df_info_new['race_adjusted'] = df_info_new['race'].map({'B':'B','W':'W','O':'O','1':'W'})

C:\Users\lucas\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [52]:
df_info_new.head()

,bioID,useFirst,nameNick,firstName,lastName,height,weight,college,birthCountry,race,nickname,top_college,foreign_birth,race_adjusted
0,abdelal01,Alaa,NaN,Alaa,Abdelnaby,82.0,240.0,Duke,EGY,B,0,1,1,B
1,abdulka01,Kareem,"Lew, Cap",Kareem,Abdul-Jabbar,85.0,225.0,UCLA,USA,B,1,1,0,B
2,abdulma01,Mahdi,Walt,Mahdi,Abdul-Rahman,74.0,185.0,UCLA,USA,B,1,1,0,B
3,abdulma02,Mahmoud,NaN,Mahmoud,Abdul-Rauf,73.0,162.0,Louisiana State,USA,B,0,0,0,B
4,abdulta01,Tariq,NaN,Tariq,Abdul-Wahad,78.0,223.0,San Jose State,FRA,B,0,0,1,B


In [53]:
df_info_new = df_info_new.drop(['college','birthCountry','race'],axis=1)

In [54]:
df_info_new.head()

,bioID,useFirst,nameNick,firstName,lastName,height,weight,nickname,top_college,foreign_birth,race_adjusted
0,abdelal01,Alaa,NaN,Alaa,Abdelnaby,82.0,240.0,0,1,1,B
1,abdulka01,Kareem,"Lew, Cap",Kareem,Abdul-Jabbar,85.0,225.0,1,1,0,B
2,abdulma01,Mahdi,Walt,Mahdi,Abdul-Rahman,74.0,185.0,1,1,0,B
3,abdulma02,Mahmoud,NaN,Mahmoud,Abdul-Rauf,73.0,162.0,0,0,0,B
4,abdulta01,Tariq,NaN,Tariq,Abdul-Wahad,78.0,223.0,0,0,1,B


## 2.8. Feature Engineering - All-Star Appearances

We will create a feature that records the number of all-star appearances for each player. 

In [55]:
df_all_star_count = pd.DataFrame(df_all_star.groupby('player_id').count()['first_name'])

In [56]:
df_all_star_count.columns = ['all_star_count']
df_all_star_count = df_all_star_count.reset_index()

In [57]:
df_all_star_count.head()

,player_id,all_star_count
0,abdulka01,19
1,abdursh01,1
2,adamsal01,1
3,adamsmi01,1
4,aguirma01,3


## 2.9. Feature Engineering - Playoff Performance

The hallmark of a successful basketball career is winning an NBA championship. Thus, we will create a feature that counts the number of championships an NBA player has won - as a player or a coach. 

In [58]:
championship_teams = df_teams[(df_teams['playoff']=='NC') | (df_teams['playoff']=='AC')]

In [59]:
championship_teams.head()

,year,lgID,tmID,franchID,confID,divID,rank,confRank,playoff,name,...,divWon,divLoss,pace,won,lost,games,min,arena,attendance,bbtmID
5,1946,NBA,PHW,GSW,NaN,ED,2,0,NC,Philadelphia Warriors,...,19,11,0,35,25,60,14575.0,Philadelphia Arena,32767,PHW
11,1947,NBA,BLB,BLB,NaN,WD,2,0,NC,Baltimore Bullets,...,10,14,0,28,20,48,11695.0,Baltimore Coliseum,0,BLT
24,1948,NBA,MNL,LAL,NaN,WD,2,0,NC,Minneapolis Lakers,...,22,8,0,44,16,60,14600.0,Minneapolis Auditorium,0,MNL
38,1949,NBA,MNL,LAL,NaN,CD,1,0,NC,Minneapolis Lakers,...,16,8,0,51,17,68,16545.0,Minneapolis Auditorium,0,MNL
55,1950,NBA,ROC,SAC,NaN,WD,2,0,NC,Rochester Royals,...,18,15,0,41,27,68,16770.0,Edgerton Park Arena,0,ROC


In order to pair each player with his respective championships, we need to first determine the last team for which each player played in every season. We will filter out duplicates of year and player, except for the last instance of a duplicate. 

In [60]:
df_stats_unique = df_stats.copy()

In [61]:
for index,row in df_stats_unique.iterrows():
    if (len(df_stats_unique[(df_stats_unique['Year']==row['Year']) & (df_stats_unique['Player']==row['Player'])])>1):
        df_stats_unique = df_stats_unique.drop(index)

In [62]:
df_stats_unique

,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1950.0,Curly Armstrong,G-F,31.0,FTW,63.0,NaN,NaN,NaN,0.368,...,0.705,NaN,NaN,NaN,176.0,NaN,NaN,NaN,217.0,458.0
1,1950.0,Cliff Barker,SG,29.0,INO,49.0,NaN,NaN,NaN,0.435,...,0.708,NaN,NaN,NaN,109.0,NaN,NaN,NaN,99.0,279.0
2,1950.0,Leo Barnhorst,SF,25.0,CHS,67.0,NaN,NaN,NaN,0.394,...,0.698,NaN,NaN,NaN,140.0,NaN,NaN,NaN,192.0,438.0
5,1950.0,Ed Bartels,F,24.0,NYK,2.0,NaN,NaN,NaN,0.376,...,0.667,NaN,NaN,NaN,0.0,NaN,NaN,NaN,2.0,4.0
6,1950.0,Ralph Beard,G,22.0,INO,60.0,NaN,NaN,NaN,0.422,...,0.762,NaN,NaN,NaN,233.0,NaN,NaN,NaN,132.0,895.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24686,2017.0,Cody Zeller,PF,24.0,CHO,62.0,58.0,1725.0,16.7,0.604,...,0.679,135.0,270.0,405.0,99.0,62.0,58.0,65.0,189.0,639.0
24687,2017.0,Tyler Zeller,C,27.0,BOS,51.0,5.0,525.0,13.0,0.508,...,0.564,43.0,81.0,124.0,42.0,7.0,21.0,20.0,61.0,178.0
24688,2017.0,Stephen Zimmerman,C,20.0,ORL,19.0,0.0,108.0,7.3,0.346,...,0.600,11.0,24.0,35.0,4.0,2.0,5.0,3.0,17.0,23.0
24689,2017.0,Paul Zipser,SF,22.0,CHI,44.0,18.0,843.0,6.9,0.503,...,0.775,15.0,110.0,125.0,36.0,15.0,16.0,40.0,78.0,240.0


In [63]:
df_stats_unique = df_stats_unique[['Year','Player','Tm']]

In [64]:
df_stats_unique[df_stats_unique['Year'].isnull()] # non-players in the database

,Year,Player,Tm
312,NaN,NaN,NaN
487,NaN,NaN,NaN
618,NaN,NaN,NaN
779,NaN,NaN,NaN
911,NaN,NaN,NaN
...,...,...,...
21678,NaN,NaN,NaN
22252,NaN,NaN,NaN
22864,NaN,NaN,NaN
23516,NaN,NaN,NaN


In [65]:
df_stats_unique = df_stats_unique[df_stats_unique['Year'].isnull()==False]

In [66]:
df_stats_unique['Year'] = df_stats_unique['Year'].astype(int)

In [67]:
df_stats_unique.head()

,Year,Player,Tm
0,1950,Curly Armstrong,FTW
1,1950,Cliff Barker,INO
2,1950,Leo Barnhorst,CHS
5,1950,Ed Bartels,NYK
6,1950,Ralph Beard,INO


We must split the player name into first name and last name in order to work with the other data source. 

In [68]:
# Veryify that all players have only two names listed
for name in df_stats_unique['Player']:
    if (len(name.split())>2):
        print('Here')

In [69]:
df_stats_unique['First'] = df_stats_unique['Player'].apply(lambda x:x.split()[0])
df_stats_unique['Last'] = df_stats_unique['Player'].apply(lambda x:x.split()[1])

In [70]:
def filter_last(name):
    if (name[-1:]=='*'):
        return name[:-1]
    else:
        return name

In [71]:
df_stats_unique['Last'] = df_stats_unique['Last'].apply(filter_last)

In [72]:
df_stats_unique

,Year,Player,Tm,First,Last
0,1950,Curly Armstrong,FTW,Curly,Armstrong
1,1950,Cliff Barker,INO,Cliff,Barker
2,1950,Leo Barnhorst,CHS,Leo,Barnhorst
5,1950,Ed Bartels,NYK,Ed,Bartels
6,1950,Ralph Beard,INO,Ralph,Beard
...,...,...,...,...,...
24686,2017,Cody Zeller,CHO,Cody,Zeller
24687,2017,Tyler Zeller,BOS,Tyler,Zeller
24688,2017,Stephen Zimmerman,ORL,Stephen,Zimmerman
24689,2017,Paul Zipser,CHI,Paul,Zipser


Now we are ready to count the number of playing and coaching championships for each player. 

In [73]:
df_champs = df_info[['bioID','firstName','lastName']]

In [74]:
df_champs.head()

,bioID,firstName,lastName
0,abdelal01,Alaa,Abdelnaby
1,abdulka01,Kareem,Abdul-Jabbar
2,abdulma01,Mahdi,Abdul-Rahman
3,abdulma02,Mahmoud,Abdul-Rauf
4,abdulta01,Tariq,Abdul-Wahad


In [75]:
championships = []
for index1,row1 in df_champs.iterrows():
    champs = 0
    if (row1['bioID'] in df_coaches['coachID'].unique()):
        for index2,row2 in df_coaches[df_coaches['coachID']==row1['bioID']].iterrows():
            if (len(championship_teams[(championship_teams['year']==row2['year']) & (championship_teams['tmID']==row2['tmID'])])>0):
                champs = champs+1
    for index3,row3 in df_stats_unique[(df_stats_unique['First']==row1['firstName']) & (df_stats_unique['Last']==row1['lastName'])].iterrows():
        if (len(championship_teams[(championship_teams['year']==row3['Year']) & (championship_teams['tmID']==row3['Tm'])])>0):
            champs = champs+1
    championships.append(champs)

In [76]:
championships[:5]

[0, 5, 0, 0, 0]

In [77]:
df_champs['championships'] = pd.Series(championships)

C:\Users\lucas\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [78]:
df_champs.head()

,bioID,firstName,lastName,championships
0,abdelal01,Alaa,Abdelnaby,0
1,abdulka01,Kareem,Abdul-Jabbar,5
2,abdulma01,Mahdi,Abdul-Rahman,0
3,abdulma02,Mahmoud,Abdul-Rauf,0
4,abdulta01,Tariq,Abdul-Wahad,0


## 2.10. Merging All Data Frames

To our advanatge, most of our feature-engineered data frames are connected through unique player IDs. This setup will allow us to join most of the data frames on IDs. 

Here are the all of the data frames that need to be merged:
- df_career_stats (does not contain player ID)
- df_coach_awards_count
- df_player_awards_count
- df_coaches_years
- df_draft_lottery
- df_hof_indicator
- df_info_new
- df_all_star_count
- df_champs

In [79]:
# Base data frame for merging the last 8 data frames since it contains all players
df_info_new

,bioID,useFirst,nameNick,firstName,lastName,height,weight,nickname,top_college,foreign_birth,race_adjusted
0,abdelal01,Alaa,NaN,Alaa,Abdelnaby,82.0,240.0,0,1,1,B
1,abdulka01,Kareem,"Lew, Cap",Kareem,Abdul-Jabbar,85.0,225.0,1,1,0,B
2,abdulma01,Mahdi,Walt,Mahdi,Abdul-Rahman,74.0,185.0,1,1,0,B
3,abdulma02,Mahmoud,NaN,Mahmoud,Abdul-Rauf,73.0,162.0,0,0,0,B
4,abdulta01,Tariq,NaN,Tariq,Abdul-Wahad,78.0,223.0,0,0,1,B
...,...,...,...,...,...,...,...,...,...,...,...
5056,rosegl01,Glen,NaN,Glen,Rose,NaN,NaN,0,0,0,NaN
5057,shephle01,Len,NaN,Len,Shepherd,NaN,NaN,0,0,0,NaN
5058,glammge01,George,NaN,George,Glammack,NaN,NaN,0,0,0,NaN
5059,eurasge01,Gene,NaN,Gene,Eurash,NaN,NaN,0,0,0,NaN


In [80]:
df_final = pd.merge(df_info_new,df_coach_awards_count,how='left',left_on='bioID',right_on='coachID')
df_final = pd.merge(df_final,df_player_awards_count,how='left',left_on='bioID',right_on='playerID')
df_final = pd.merge(df_final,df_coaches_years,how='left',left_on='bioID',right_on='playerID')
df_final = pd.merge(df_final,df_draft_lottery,how='left',left_on='bioID',right_on='playerID')
df_final = pd.merge(df_final,df_hof_indicator,how='left',left_on='bioID',right_on='bioID')
df_final = pd.merge(df_final,df_all_star_count,how='left',left_on='bioID',right_on='player_id')
df_final = pd.merge(df_final,df_champs,how='left',left_on='bioID',right_on='bioID')

In [81]:
df_final

,bioID,useFirst,nameNick,firstName_x,lastName_x,height,weight,nickname,top_college,foreign_birth,...,playerID_y,years_coaching,playerID,lottery,hof,player_id,all_star_count,firstName_y,lastName_y,championships
0,abdelal01,Alaa,NaN,Alaa,Abdelnaby,82.0,240.0,0,1,1,...,abdelal01,0,abdelal01,0.0,0,NaN,NaN,Alaa,Abdelnaby,0
1,abdulka01,Kareem,"Lew, Cap",Kareem,Abdul-Jabbar,85.0,225.0,1,1,0,...,abdulka01,0,abdulka01,1.0,1,abdulka01,19.0,Kareem,Abdul-Jabbar,5
2,abdulma01,Mahdi,Walt,Mahdi,Abdul-Rahman,74.0,185.0,1,1,0,...,abdulma01,0,abdulma01,0.0,0,NaN,NaN,Mahdi,Abdul-Rahman,0
3,abdulma02,Mahmoud,NaN,Mahmoud,Abdul-Rauf,73.0,162.0,0,0,0,...,abdulma02,0,abdulma02,1.0,0,NaN,NaN,Mahmoud,Abdul-Rauf,0
4,abdulta01,Tariq,NaN,Tariq,Abdul-Wahad,78.0,223.0,0,0,1,...,abdulta01,0,abdulta01,1.0,0,NaN,NaN,Tariq,Abdul-Wahad,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5056,rosegl01,Glen,NaN,Glen,Rose,NaN,NaN,0,0,0,...,rosegl01,1,NaN,NaN,0,NaN,NaN,Glen,Rose,0
5057,shephle01,Len,NaN,Len,Shepherd,NaN,NaN,0,0,0,...,shephle01,1,NaN,NaN,0,NaN,NaN,Len,Shepherd,0
5058,glammge01,George,NaN,George,Glammack,NaN,NaN,0,0,0,...,glammge01,1,NaN,NaN,0,NaN,NaN,George,Glammack,0
5059,eurasge01,Gene,NaN,Gene,Eurash,NaN,NaN,0,0,0,...,eurasge01,1,NaN,NaN,0,NaN,NaN,Gene,Eurash,0


Let us clean up the columns by removing duplicated information. 

In [82]:
df_final.columns

Index(['bioID', 'useFirst', 'nameNick', 'firstName_x', 'lastName_x', 'height',
       'weight', 'nickname', 'top_college', 'foreign_birth', 'race_adjusted',
       'coachID', 'award_count', 'playerID_x', 'all_team_awards',
       'league_awards', 'playerID_y', 'years_coaching', 'playerID', 'lottery',
       'hof', 'player_id', 'all_star_count', 'firstName_y', 'lastName_y',
       'championships'],
      dtype='object')

In [83]:
df_final = df_final.drop(['coachID','playerID_x','playerID_y','playerID','player_id','firstName_y','lastName_y'],axis=1)
df_final.columns = ['bioID','useFirst','nameNick','firstName','lastName','height','weight','nickname','top_college','foreign_birth','race_adjusted','award_count','all_team_awards','league_awards','years_coaching','lottery','hof','all_star_count','championships']

In [84]:
df_final

,bioID,useFirst,nameNick,firstName,lastName,height,weight,nickname,top_college,foreign_birth,race_adjusted,award_count,all_team_awards,league_awards,years_coaching,lottery,hof,all_star_count,championships
0,abdelal01,Alaa,NaN,Alaa,Abdelnaby,82.0,240.0,0,1,1,B,NaN,NaN,NaN,0,0.0,0,NaN,0
1,abdulka01,Kareem,"Lew, Cap",Kareem,Abdul-Jabbar,85.0,225.0,1,1,0,B,NaN,26.0,9.0,0,1.0,1,19.0,5
2,abdulma01,Mahdi,Walt,Mahdi,Abdul-Rahman,74.0,185.0,1,1,0,B,NaN,NaN,NaN,0,0.0,0,NaN,0
3,abdulma02,Mahmoud,NaN,Mahmoud,Abdul-Rauf,73.0,162.0,0,0,0,B,NaN,0.0,1.0,0,1.0,0,NaN,0
4,abdulta01,Tariq,NaN,Tariq,Abdul-Wahad,78.0,223.0,0,0,1,B,NaN,NaN,NaN,0,1.0,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5056,rosegl01,Glen,NaN,Glen,Rose,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,1,NaN,0,NaN,0
5057,shephle01,Len,NaN,Len,Shepherd,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,1,NaN,0,NaN,0
5058,glammge01,George,NaN,George,Glammack,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,1,NaN,0,NaN,0
5059,eurasge01,Gene,NaN,Gene,Eurash,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,1,NaN,0,NaN,0


Now we must merge df_career_stats with df_final. These data frames come from different data sources, so we must join on player first name and last name. 

In [85]:
df_career_stats

,Player,G_sum,GS_sum,MP_sum,FG_sum,FGA_sum,3P_sum,3PA_sum,2P_sum,2PA_sum,...,BPM_mean,FG%_mean,3P%_mean,2P%_mean,eFG%_mean,FT%_mean,OWS_sum,DWS_sum,WS_sum,VORP_sum
0,A.C. Green,1361.0,905.0,39044.0,4778.0,9686.0,125.0,509.0,4653.0,9177.0,...,-0.083333,0.487667,0.146833,0.501889,0.493167,0.708056,60.7,43.5,104.2,23.0
1,A.J. Bramlett,8.0,0.0,61.0,4.0,21.0,0.0,0.0,4.0,21.0,...,-16.800000,0.190000,NaN,0.190000,0.190000,NaN,-0.2,0.1,-0.2,-0.2
2,A.J. English,151.0,18.0,3108.0,617.0,1418.0,9.0,65.0,608.0,1353.0,...,-5.100000,0.436000,0.136500,0.450500,0.438500,0.774000,-0.4,1.4,1.1,-2.4
3,A.J. Guyton,80.0,14.0,1246.0,166.0,440.0,73.0,193.0,93.0,247.0,...,-11.000000,0.255667,0.255000,0.254000,0.310667,0.824000,0.3,-0.1,0.3,-1.0
4,A.J. Price,150.0,3.0,2228.0,318.0,848.0,127.0,411.0,191.0,437.0,...,-2.366667,0.368333,0.305000,0.427667,0.442667,0.755667,0.2,2.0,2.2,-0.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3525,Zeljko Rebraca,239.0,24.0,3553.0,522.0,1003.0,0.0,0.0,522.0,1003.0,...,-3.485714,0.505429,NaN,0.505429,0.505429,0.747286,3.9,4.8,8.7,-0.4
3526,Zelmo Beaty*,570.0,0.0,18348.0,3396.0,7237.0,0.0,0.0,3396.0,7237.0,...,-2.300000,0.463500,NaN,0.463500,0.463500,0.751750,36.9,21.3,58.1,-0.8
3527,Zendon Hamilton,146.0,15.0,1609.0,183.0,414.0,0.0,0.0,183.0,414.0,...,-8.950000,0.370125,NaN,0.370125,0.370125,0.679250,1.4,1.5,3.2,-1.3
3528,Zoran Planinic,148.0,10.0,1584.0,197.0,486.0,37.0,128.0,160.0,358.0,...,-4.133333,0.406667,0.296000,0.447000,0.444333,0.675667,-0.6,2.1,1.6,-0.9


In [86]:
df_career_stats['First'] = df_career_stats['Player'].apply(lambda x:x.split()[0])
df_career_stats['Last'] = df_career_stats['Player'].apply(lambda x:x.split()[1])
df_career_stats['Last'] = df_career_stats['Last'].apply(filter_last)
df_career_stats['Player'] = df_career_stats['Player'].apply(filter_last) # function works on column with full names too

In [87]:
df_career_stats.head()

,Player,G_sum,GS_sum,MP_sum,FG_sum,FGA_sum,3P_sum,3PA_sum,2P_sum,2PA_sum,...,3P%_mean,2P%_mean,eFG%_mean,FT%_mean,OWS_sum,DWS_sum,WS_sum,VORP_sum,First,Last
0,A.C. Green,1361.0,905.0,39044.0,4778.0,9686.0,125.0,509.0,4653.0,9177.0,...,0.146833,0.501889,0.493167,0.708056,60.7,43.5,104.2,23.0,A.C.,Green
1,A.J. Bramlett,8.0,0.0,61.0,4.0,21.0,0.0,0.0,4.0,21.0,...,NaN,0.190000,0.190000,NaN,-0.2,0.1,-0.2,-0.2,A.J.,Bramlett
2,A.J. English,151.0,18.0,3108.0,617.0,1418.0,9.0,65.0,608.0,1353.0,...,0.136500,0.450500,0.438500,0.774000,-0.4,1.4,1.1,-2.4,A.J.,English
3,A.J. Guyton,80.0,14.0,1246.0,166.0,440.0,73.0,193.0,93.0,247.0,...,0.255000,0.254000,0.310667,0.824000,0.3,-0.1,0.3,-1.0,A.J.,Guyton
4,A.J. Price,150.0,3.0,2228.0,318.0,848.0,127.0,411.0,191.0,437.0,...,0.305000,0.427667,0.442667,0.755667,0.2,2.0,2.2,-0.3,A.J.,Price


In [88]:
df_final1 = pd.merge(df_career_stats,df_final,how='inner',left_on=['First','Last'],right_on=['firstName','lastName'])
df_final2 = pd.merge(df_career_stats,df_final,how='inner',left_on=['First','Last'],right_on=['useFirst','lastName'])
df_final3 = pd.merge(df_career_stats,df_final,how='inner',left_on=['First','Last'],right_on=['nameNick','lastName'])

In [89]:
df_final_agg = pd.concat([df_final1,df_final2,df_final3])

In [90]:
df_final = df_final_agg.drop_duplicates()

In [91]:
df_final

,Player,G_sum,GS_sum,MP_sum,FG_sum,FGA_sum,3P_sum,3PA_sum,2P_sum,2PA_sum,...,foreign_birth,race_adjusted,award_count,all_team_awards,league_awards,years_coaching,lottery,hof,all_star_count,championships
0,A.C. Green,1361.0,905.0,39044.0,4778.0,9686.0,125.0,509.0,4653.0,9177.0,...,0,B,NaN,1.0,0.0,0,0.0,0,1.0,3
1,A.J. Wynder,6.0,0.0,39.0,3.0,12.0,0.0,1.0,3.0,11.0,...,0,B,NaN,NaN,NaN,0,NaN,0,NaN,0
2,Aaron Brooks,331.0,141.0,8093.0,1424.0,3513.0,498.0,1415.0,926.0,2098.0,...,0,B,NaN,0.0,1.0,0,0.0,0,NaN,0
3,Aaron Gray,271.0,65.0,3294.0,407.0,795.0,0.0,2.0,407.0,793.0,...,0,W,NaN,NaN,NaN,0,0.0,0,NaN,0
4,Aaron James,356.0,0.0,7671.0,1609.0,3347.0,0.0,0.0,1609.0,3347.0,...,0,B,NaN,NaN,NaN,0,0.0,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
812,Patty Mills,90.0,3.0,1082.0,208.0,482.0,73.0,193.0,135.0,289.0,...,1,B,NaN,NaN,NaN,0,0.0,0,NaN,0
879,Rick Wilson,66.0,0.0,648.0,83.0,211.0,0.0,0.0,83.0,211.0,...,0,B,NaN,NaN,NaN,0,0.0,0,NaN,0
1071,Walt Budko,193.0,0.0,1126.0,460.0,1356.0,0.0,0.0,460.0,1356.0,...,0,W,NaN,NaN,NaN,1,1.0,0,NaN,0
1076,Wes Matthews,564.0,107.0,11959.0,1909.0,3969.0,42.0,189.0,1867.0,3780.0,...,0,B,NaN,NaN,NaN,0,NaN,0,NaN,0


In [92]:
df_career_stats # should have slightly more rows than above

,Player,G_sum,GS_sum,MP_sum,FG_sum,FGA_sum,3P_sum,3PA_sum,2P_sum,2PA_sum,...,3P%_mean,2P%_mean,eFG%_mean,FT%_mean,OWS_sum,DWS_sum,WS_sum,VORP_sum,First,Last
0,A.C. Green,1361.0,905.0,39044.0,4778.0,9686.0,125.0,509.0,4653.0,9177.0,...,0.146833,0.501889,0.493167,0.708056,60.7,43.5,104.2,23.0,A.C.,Green
1,A.J. Bramlett,8.0,0.0,61.0,4.0,21.0,0.0,0.0,4.0,21.0,...,NaN,0.190000,0.190000,NaN,-0.2,0.1,-0.2,-0.2,A.J.,Bramlett
2,A.J. English,151.0,18.0,3108.0,617.0,1418.0,9.0,65.0,608.0,1353.0,...,0.136500,0.450500,0.438500,0.774000,-0.4,1.4,1.1,-2.4,A.J.,English
3,A.J. Guyton,80.0,14.0,1246.0,166.0,440.0,73.0,193.0,93.0,247.0,...,0.255000,0.254000,0.310667,0.824000,0.3,-0.1,0.3,-1.0,A.J.,Guyton
4,A.J. Price,150.0,3.0,2228.0,318.0,848.0,127.0,411.0,191.0,437.0,...,0.305000,0.427667,0.442667,0.755667,0.2,2.0,2.2,-0.3,A.J.,Price
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3525,Zeljko Rebraca,239.0,24.0,3553.0,522.0,1003.0,0.0,0.0,522.0,1003.0,...,NaN,0.505429,0.505429,0.747286,3.9,4.8,8.7,-0.4,Zeljko,Rebraca
3526,Zelmo Beaty,570.0,0.0,18348.0,3396.0,7237.0,0.0,0.0,3396.0,7237.0,...,NaN,0.463500,0.463500,0.751750,36.9,21.3,58.1,-0.8,Zelmo,Beaty
3527,Zendon Hamilton,146.0,15.0,1609.0,183.0,414.0,0.0,0.0,183.0,414.0,...,NaN,0.370125,0.370125,0.679250,1.4,1.5,3.2,-1.3,Zendon,Hamilton
3528,Zoran Planinic,148.0,10.0,1584.0,197.0,486.0,37.0,128.0,160.0,358.0,...,0.296000,0.447000,0.444333,0.675667,-0.6,2.1,1.6,-0.9,Zoran,Planinic


Since the number of rows in df_career_stats is lower than the number of rows in df_final, we still have duplicate players in df_final. These duplicates are likely to be the result of multiple players having the same name. We will drop all of these duplicate "players" to eliminate any confusion.

In [93]:
df_final[df_final['Player'].duplicated()]

,Player,G_sum,GS_sum,MP_sum,FG_sum,FGA_sum,3P_sum,3PA_sum,2P_sum,2PA_sum,...,foreign_birth,race_adjusted,award_count,all_team_awards,league_awards,years_coaching,lottery,hof,all_star_count,championships
46,Alfred McGuire,191.0,0.0,2966.0,251.0,663.0,0.0,0.0,251.0,663.0,...,0,W,NaN,NaN,NaN,0,0.0,0,NaN,0
123,Anthony Tucker,62.0,13.0,982.0,96.0,210.0,0.0,1.0,96.0,209.0,...,0,B,NaN,NaN,NaN,0,0.0,0,NaN,0
283,Cedric Henderson,276.0,136.0,6214.0,791.0,1813.0,6.0,43.0,785.0,1770.0,...,0,B,NaN,NaN,NaN,0,0.0,0,NaN,0
298,Charles Davis,497.0,34.0,6050.0,1171.0,2623.0,21.0,139.0,1150.0,2484.0,...,0,B,NaN,NaN,NaN,0,0.0,0,NaN,1
302,Charles Johnson,592.0,0.0,11271.0,2172.0,5088.0,0.0,0.0,2172.0,5088.0,...,0,W,NaN,NaN,NaN,0,NaN,0,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
327,Don Smith,54.0,0.0,538.0,131.0,321.0,0.0,0.0,131.0,321.0,...,0,W,NaN,NaN,NaN,0,NaN,0,NaN,0
603,Johnny Orr,68.0,0.0,0.0,80.0,236.0,0.0,0.0,80.0,236.0,...,0,W,NaN,NaN,NaN,0,NaN,0,NaN,0
879,Rick Wilson,66.0,0.0,648.0,83.0,211.0,0.0,0.0,83.0,211.0,...,0,B,NaN,NaN,NaN,0,0.0,0,NaN,0
1076,Wes Matthews,564.0,107.0,11959.0,1909.0,3969.0,42.0,189.0,1867.0,3780.0,...,0,B,NaN,NaN,NaN,0,NaN,0,NaN,0


In [94]:
# All repeated names in df_final
repeated_names = df_final[df_final['Player'].duplicated()]['Player'].unique()

In [95]:
# Example of a repeated name with Sr. and Jr. 
df_final[df_final['Player']=='Wes Matthews']

,Player,G_sum,GS_sum,MP_sum,FG_sum,FGA_sum,3P_sum,3PA_sum,2P_sum,2PA_sum,...,foreign_birth,race_adjusted,award_count,all_team_awards,league_awards,years_coaching,lottery,hof,all_star_count,championships
3391,Wes Matthews,564.0,107.0,11959.0,1909.0,3969.0,42.0,189.0,1867.0,3780.0,...,0,B,NaN,NaN,NaN,0,1.0,0,NaN,0
1076,Wes Matthews,564.0,107.0,11959.0,1909.0,3969.0,42.0,189.0,1867.0,3780.0,...,0,B,NaN,NaN,NaN,0,NaN,0,NaN,0


In [96]:
indices = []
for index,row in df_final.iterrows():
    if (row['Player'] in repeated_names):
        indices.append(index)
df_final = df_final.drop(indices)

In [97]:
df_final

,Player,G_sum,GS_sum,MP_sum,FG_sum,FGA_sum,3P_sum,3PA_sum,2P_sum,2PA_sum,...,foreign_birth,race_adjusted,award_count,all_team_awards,league_awards,years_coaching,lottery,hof,all_star_count,championships
0,A.C. Green,1361.0,905.0,39044.0,4778.0,9686.0,125.0,509.0,4653.0,9177.0,...,0,B,NaN,1.0,0.0,0,0.0,0,1.0,3
1,A.J. Wynder,6.0,0.0,39.0,3.0,12.0,0.0,1.0,3.0,11.0,...,0,B,NaN,NaN,NaN,0,NaN,0,NaN,0
2,Aaron Brooks,331.0,141.0,8093.0,1424.0,3513.0,498.0,1415.0,926.0,2098.0,...,0,B,NaN,0.0,1.0,0,0.0,0,NaN,0
3,Aaron Gray,271.0,65.0,3294.0,407.0,795.0,0.0,2.0,407.0,793.0,...,0,W,NaN,NaN,NaN,0,0.0,0,NaN,0
4,Aaron James,356.0,0.0,7671.0,1609.0,3347.0,0.0,0.0,1609.0,3347.0,...,0,B,NaN,NaN,NaN,0,0.0,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
676,Magic Johnson,906.0,763.0,33245.0,6211.0,11951.0,325.0,1074.0,5886.0,10877.0,...,0,B,NaN,10.0,6.0,1,1.0,1,3.0,0
737,Mike Holton,353.0,122.0,6348.0,873.0,1980.0,29.0,121.0,844.0,1859.0,...,0,B,NaN,NaN,NaN,0,0.0,0,NaN,0
772,Mo Williams,647.0,488.0,19680.0,3360.0,7689.0,876.0,2306.0,2484.0,5383.0,...,0,B,NaN,NaN,NaN,0,0.0,0,1.0,0
812,Patty Mills,90.0,3.0,1082.0,208.0,482.0,73.0,193.0,135.0,289.0,...,1,B,NaN,NaN,NaN,0,0.0,0,NaN,0


It is important to consider that our data-merging process creates a loss of around 7% of data. This consequence is disappointing, but it represents the reality of using data from different sources. Hopefully, universal player IDs will be assigned to NBA players in the future so that this problem can be circumvented. 

## 2.11. Separating Retired and Non-Retired Players

The last part of this data preparation before cleaning is to ensure that we are only including retired players. It would not make sense for us to perform modeling on current players since they cannot be in the hall of fame. 

In [98]:
# Equalizing name column of df_finish to player column of df_final
df_finish['name'] = df_finish['name'].apply(filter_last)

In [99]:
# Data frame with last year played
df_finish.head()

,name,year_start,year_end,position,height,weight,birth_date,college
0,Alaa Abdelnaby,1991,1995,F-C,6-10,240.0,"June 24, 1968",Duke University
1,Zaid Abdul-Aziz,1969,1978,C-F,6-9,235.0,"April 7, 1946",Iowa State University
2,Kareem Abdul-Jabbar,1970,1989,C,7-2,225.0,"April 16, 1947","University of California, Los Angeles"
3,Mahmoud Abdul-Rauf,1991,2001,G,6-1,162.0,"March 9, 1969",Louisiana State University
4,Tariq Abdul-Wahad,1998,2003,F,6-6,223.0,"November 3, 1974",San Jose State University


In [100]:
len(df_finish[df_finish['name'].duplicated()])

50

There will be instances where some players in df_final have more than one instance in df_finish. These players in df_final bypassed our initial duplicate filtration, so we will drop them at this point. 

In [101]:
df_final = df_final.reset_index(drop=True)

In [102]:
dropped_instances = []
non_retired_players = []

In [103]:
for index,row in df_final.iterrows():
    if (len(df_finish[df_finish['name']==row['Player']])>1):
        dropped_instances.append(index)
    else:
        if (len(df_finish[df_finish['name']==row['Player']])>0):
            if (df_finish[df_finish['name']==row['Player']]['year_end'].iloc[0]>2012):
                non_retired_players.append(index)
        else: # no retirement information on player
            dropped_instances.append(index)

In [104]:
df_final_active = df_final.iloc[non_retired_players]

In [105]:
df_final = df_final.drop(dropped_instances)
df_final = df_final.drop(non_retired_players)

In [106]:
df_final = df_final.reset_index(drop=True)
df_final_active = df_final_active.reset_index(drop=True)

In [107]:
# Final pre-cleaned data frame
df_final

,Player,G_sum,GS_sum,MP_sum,FG_sum,FGA_sum,3P_sum,3PA_sum,2P_sum,2PA_sum,...,foreign_birth,race_adjusted,award_count,all_team_awards,league_awards,years_coaching,lottery,hof,all_star_count,championships
0,A.C. Green,1361.0,905.0,39044.0,4778.0,9686.0,125.0,509.0,4653.0,9177.0,...,0,B,NaN,1.0,0.0,0,0.0,0,1.0,3
1,A.J. Wynder,6.0,0.0,39.0,3.0,12.0,0.0,1.0,3.0,11.0,...,0,B,NaN,NaN,NaN,0,NaN,0,NaN,0
2,Aaron James,356.0,0.0,7671.0,1609.0,3347.0,0.0,0.0,1609.0,3347.0,...,0,B,NaN,NaN,NaN,0,0.0,0,NaN,0
3,Aaron McKie,957.0,330.0,22594.0,2554.0,5923.0,441.0,1275.0,2113.0,4648.0,...,0,B,NaN,0.0,1.0,0,0.0,0,NaN,0
4,Aaron Miles,19.0,0.0,118.0,6.0,18.0,0.0,0.0,6.0,18.0,...,0,B,NaN,NaN,NaN,0,NaN,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2885,Em Bryant,566.0,0.0,11451.0,1501.0,3665.0,0.0,0.0,1501.0,3665.0,...,0,B,NaN,NaN,NaN,0,0.0,0,NaN,0
2886,Gar Heard,954.0,0.0,23870.0,3588.0,8602.0,0.0,9.0,3588.0,8593.0,...,0,B,NaN,NaN,NaN,2,0.0,0,NaN,0
2887,Magic Johnson,906.0,763.0,33245.0,6211.0,11951.0,325.0,1074.0,5886.0,10877.0,...,0,B,NaN,10.0,6.0,1,1.0,1,3.0,0
2888,Mike Holton,353.0,122.0,6348.0,873.0,1980.0,29.0,121.0,844.0,1859.0,...,0,B,NaN,NaN,NaN,0,0.0,0,NaN,0


## 2.12. Saving Pre-Cleaned Data Frames

In [108]:
df_final.to_csv('./Data/final_not_cleaned.csv')

In [109]:
df_final_active.to_csv('./Data/final_active_not_cleaned.csv')

## 2.13. Further Steps

In the next section, we will conduct exploratory data analysis on the final data frame. This procedure will involve determining what to do with statistical null values and selecting which features to use in the model. 

Furthermore, it is likely that there is a high degree of collinearity between some of the statistical features. We will investigate column removal, principal component analysis, and other appropriate techniques for minimizing collinearity. 